<a href="https://colab.research.google.com/github/sidesbutgithub/GlassLab/blob/main/ChangingPTTW_GlassLab_Sidney.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project one is to fit the simplest model EUT to data
will use curve_fit from scipy and look into pyomo for abstract functions


CURRENT PROGRESS

model -creation/predictions are done.

Data sorting is done

Now we are fitting

We have used scipy.optimize to optimize our params
we are using get_loss_one as the objective function which finds the mean squared error averaged between the preds and actuals.

We have used nelder mead, powell, cobyla. These however have produced bad results for the top 10 participants.

We have tried longer optimization algorithms such as binhopping and shgo however they are taking over 20 minutes per participant without sign of slowing down therefore we are not going to use them.

Differential evolution seems to be working the best. We have implemetned a constraint over the optimzation to make sure that the values stay within the bounds and constrains outlined in the report Glass gave us.

The runtime seems to be around 13-15 minutes per participant.
we can try and optimize the algorithm by closing all tabs, and modifying the parameters however trying a gridsearch or parameters search for this tuning may take too long however if optimal parameters can be broadcast to all participants it might be worth it.

We are yet to try dual annealing or the DIRECT method from scipy.optimize and we haven't attempted the recursive linear fit outlined in yudelson's code.
Yudelson's greedy fit I think will take too long, however we haven't tried running it for over 10 minutes yet.

Here are the DE results for partiicpant 1 (p=0).

array**([a: 0.25019451, b: 0.26774997,l: 1.19726157, g: 0.99461636]**)
although this is vastly different than the actual params the yielded predictions are the same.

predictions for participant 7
array([0.89365145, 0.92065334, 1.38266249, 0.84074097])

8/18/23
We fit the first 4 participants using the differential evolution

it took an estimated time of 48 minutes 12 per participant.
While participants 1,3 and 4 resulted in d2a's of less than 11 percent
participant 2 had a fit that was completely off and missed the prediction on day 4.

To work on next we try and change the preciscion or how much the model is jumping by to make the search space smalleer. For example, we can make it only search by the hundreths.

Another thing we can do is we can run the fitting with low iterations and then go back and re fit the participants that had bad results until the algo converges.

One pattern that might be present, is that when alhpa and beta are low, the model might not work.

To improve preciscion, we can increase the iteration size as well as decrease the custom tolerance for the d2a error, making it so that it will only stop once it's really small.

Instructions for running:

run all the neccescary functions and all (few important ones you might miss are the d2a_error, creating soldOnly).
Make sure you scroll down to where I say first4=[ array, array ,array, array ...]
and run that since the fitting loop here needs it
make sure to initialize the fiveto8 list as empty
make sure to put the callbacks into the call_backs list
then just run the loop, it should take quite some time as I tripled the maxiter


WANDB KEY : 61cca3bef0200dc37d895eb5a336add7f668eb69


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
#from penis import big

In [ ]:
NEIGHBORS_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/'

In [ ]:
POLISHED_DIRECT_PATH = '/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/'

In [ ]:
DIRECT_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/DIRECT fits/'  # THIS IS SPECIFIC TO MY DRIVE, YOU WILL HAVE TO CREATE YOUR OWN FOLDER AND THUS CHANGE THE PATH NAME

In [ ]:
DIFEV_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits/'

In [ ]:
BASIN_HOPPING_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/BASIN_HOPPING_FITS/'

In [ ]:
SHGO_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT fits/SHGO_FITS/'

In [ ]:
EXHUASTIVE_SAVE_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Exhuastive fits/'

In [ ]:
BOBBY_PATH='/content/drive/MyDrive/Glass Lab Data/PT_TW fits/BOBBY FITS/'

In [ ]:
POLISH0_PATH='/'

In [ ]:
import pickle
from datetime import date
def save_fit(fit, participant,algo):
  tdy=date.today()
  sp=""
  if algo=="direct":
    sp=DIRECT_SAVE_PATH
  elif algo=="diffev":
    sp=DIFEV_SAVE_PATH
  elif algo=="basin_hopping":
    sp=BASIN_HOPPING_SAVE_PATH
  elif algo=="shgo":
    sp=SHGO_SAVE_PATH
  elif algo=="brute":
    sp=EXHUASTIVE_SAVE_PATH
  elif algo=='bobby':
    sp=BOBBY_PATH
  elif algo=='neighbors':
    sp=NEIGHBORS_PATH
  elif algo=='polished_direct':
    sp=POLISHED_DIRECT_PATH



  fname=sp+"participant " + str(participant)+" "+tdy.strftime("%B %d, %Y")+".pickle"
  pickle_out=open(fname,"wb")
  pickle.dump(fit,pickle_out)
  pickle_out.close()



In [ ]:
def load_fit(path):
  pickle_in=open(path,"rb") # put the path here
  fit=pickle.load(pickle_in)
  return fit


In [ ]:
p = {
    1: 0.03,
    2: 0.06,
    3: 0.09,
    4: 0.12,
    5 : 0.14,
    6: 0.11,
    7: 0.09,
    8: 0.08,
    9: 0.07,
    10: 0.06,
    11: 0.05,
    12: 0.04,
    13: 0.03,
    14: 0.02,
    15: 0.01
}
I=15

In [ ]:




def EUTv1(day, i_d, n, stored ):
    s=0
    for j in p.keys:
      s+=(j*(p[j]*10))

    util = n*i_d + (stored-n)*s
    return util




In [ ]:


def cutoff_EUT(x):
    if x >= 32:
        return 14
    elif x in range(17, 32):
        return 13
    elif x in range(10, 17):
        return 12
    elif x in range(7, 10):
        return 11
    elif x in range(5, 7):
        return 10
    elif x == 4:
        return 9
    elif x == 3:
        return 8
    elif x == 2:
        return 7
    elif x == 1:
        return 1




In [ ]:

def h_prob(day_f, actual_day): # This returns the probability of a sale on the day (d=h) where day is the index variable used in the util represented by f and actual day is the actual day used in the util represented by "day"

  h_rem= 1
  for h in range(actual_day-1, day_f+1, -1):
    h_rem*=sum(p[k] for k in range(1,cutoff_EUT(h)-1))

  h_price=sum(p[j]*j for j in range(cutoff_EUT(day_f),I))
  return (h_price*h_rem)

In [ ]:
def EUTv2(day, i_d, stored, n):



  prob_j = sum(h_prob(f,day) for f in range(day-2, 1,-1)) # edit: report just has a big formattinng issue, all is good
  s=0
  for j in range(cutoff_t(day-1), I):
      s+=j*(p[j])


  util= n*i_d + (stored-n)*(s+ prob_j )

  return util




The independent variables in this experiment are i_d, p and n


In [ ]:
all_data= pd.read_csv('/content/drive/MyDrive/Glass Lab Data/All Data EUT 230519.csv')

In [ ]:

SellData= all_data.copy()

In [ ]:
columns=list(SellData.columns)
columns

['Subject',
 'Day',
 'Stored68',
 'Price68',
 'Sold68',
 'Predicted68',
 'Stored67',
 'Price67',
 'Sold67',
 'Predicted67',
 'Stored66',
 'Price66',
 'Sold66',
 'Predicted66',
 'Stored65',
 'Price65',
 'Sold65',
 'Predicted65',
 'Stored64',
 'Price64',
 'Sold64',
 'Predicted64',
 'Stored63',
 'Price63',
 'Sold63',
 'Predicted63',
 'Stored62',
 'Price62',
 'Sold62',
 'Predicted62',
 'Stored61',
 'Price61',
 'Sold61',
 'Predicted61',
 'Stored60',
 'Price60',
 'Sold60',
 'Predicted60',
 'Stored59',
 'Price59',
 'Sold59',
 'Predicted59',
 'Stored58',
 'Price58',
 'Sold58',
 'Predicted58',
 'Stored57',
 'Price57',
 'Sold57',
 'Predicted57',
 'Stored56',
 'Price56',
 'Sold56',
 'Predicted56',
 'Stored55',
 'Price55',
 'Sold55',
 'Predicted55',
 'Stored54',
 'Price54',
 'Sold54',
 'Predicted54',
 'Stored53',
 'Price53',
 'Sold53',
 'Predicted53',
 'Stored52',
 'Price52',
 'Sold52',
 'Predicted52',
 'Stored51',
 'Price51',
 'Sold51',
 'Predicted51',
 'Stored50',
 'Price50',
 'Sold50',
 'Predic

In [ ]:
for string in columns:
  if "Predicted"  in string:
      columns.remove(string)
print(columns)

['Subject', 'Day', 'Stored68', 'Price68', 'Sold68', 'Stored67', 'Price67', 'Sold67', 'Stored66', 'Price66', 'Sold66', 'Stored65', 'Price65', 'Sold65', 'Stored64', 'Price64', 'Sold64', 'Stored63', 'Price63', 'Sold63', 'Stored62', 'Price62', 'Sold62', 'Stored61', 'Price61', 'Sold61', 'Stored60', 'Price60', 'Sold60', 'Stored59', 'Price59', 'Sold59', 'Stored58', 'Price58', 'Sold58', 'Stored57', 'Price57', 'Sold57', 'Stored56', 'Price56', 'Sold56', 'Stored55', 'Price55', 'Sold55', 'Stored54', 'Price54', 'Sold54', 'Stored53', 'Price53', 'Sold53', 'Stored52', 'Price52', 'Sold52', 'Stored51', 'Price51', 'Sold51', 'Stored50', 'Price50', 'Sold50', 'Stored49', 'Price49', 'Sold49', 'Stored48', 'Price48', 'Sold48', 'Stored47', 'Price47', 'Sold47', 'Stored46', 'Price46', 'Sold46', 'Stored45', 'Price45', 'Sold45', 'Stored44', 'Price44', 'Sold44', 'Stored43', 'Price43', 'Sold43', 'Stored42', 'Price42', 'Sold42', 'Stored41', 'Price41', 'Sold41', 'Stored40', 'Price40', 'Sold40', 'Stored39', 'Price39', '

In [ ]:
error_cols=columns[-10:]

In [ ]:
columns.remove( 'OEHold')
columns.remove( 'OHold')

columns.remove('OESold')
columns.remove('OSold')
columns.remove( 'MSold')

columns.remove( 'Diff')
columns.remove( 'Tot')
columns.remove( 'MDev')
columns.remove( 'MDev0')
columns.remove('Tot0')



In [ ]:
SellData=all_data[columns]
SellData
error_data=all_data[error_cols]

In [ ]:
df_days=[]
df_start=SellData.iloc[:,[0]]
splitting_df= SellData.drop("Subject", axis=1)


In [ ]:
def split(df):
  cols=list(df.columns)
  stored_cols=[x for x in cols if ("Stored" in x)]
 #for i in DAYS:
  #cols_for_day=[]

  for name in stored_cols:
    print("Stored+ day: ", name)

    stored_i=cols.index(name)
    print("stored index",stored_i)

    new_df= df.iloc[:, list(range(stored_i,stored_i+3))]
    print("length of new df : ", len(new_df.columns))
    print("columns of new df : ", list(new_df.columns))
    new_df= pd.concat([df_start,new_df], axis=1)
    df_days.insert(stored_i, new_df)
  print("length of df_days: ", len(df_days))




In [ ]:
split(splitting_df)

Stored+ day:  Stored68
stored index 1
length of new df :  3
columns of new df :  ['Stored68', 'Price68', 'Sold68']
Stored+ day:  Stored67
stored index 4
length of new df :  3
columns of new df :  ['Stored67', 'Price67', 'Sold67']
Stored+ day:  Stored66
stored index 7
length of new df :  3
columns of new df :  ['Stored66', 'Price66', 'Sold66']
Stored+ day:  Stored65
stored index 10
length of new df :  3
columns of new df :  ['Stored65', 'Price65', 'Sold65']
Stored+ day:  Stored64
stored index 13
length of new df :  3
columns of new df :  ['Stored64', 'Price64', 'Sold64']
Stored+ day:  Stored63
stored index 16
length of new df :  3
columns of new df :  ['Stored63', 'Price63', 'Sold63']
Stored+ day:  Stored62
stored index 19
length of new df :  3
columns of new df :  ['Stored62', 'Price62', 'Sold62']
Stored+ day:  Stored61
stored index 22
length of new df :  3
columns of new df :  ['Stored61', 'Price61', 'Sold61']
Stored+ day:  Stored60
stored index 25
length of new df :  3
columns of new

In [ ]:
df_days.reverse()

Not really. For EUT, all you are doing is calculating the value of the equation each day when a unit is sold versus when a unit is not sold. If the value of the equation is higher when a unit is not old. If the value of the equation is higher when a unit is sold then the predicted number of units sold is equal to the number of units stored. If the value of the equation is higher when a unit is not sold then the predicted number of units sold for that day is 0.


In [ ]:
DAYS=list(range(0,68))

Alright now we are going to be predicting with the PT model. This has parameters and multilpe equation variations. It's prospect theory so a simple gains vs losses.

The prospective value of a sale on day d is determined by the probability that a sale would be made at a larger or smaller price on a subsequent day. If a sale is not made for id, the likelihood of a subsequent sale on day d+v at a smaller value implies that a sale on day d for id would be a gain and the likelihood of a subsequent sale on day d+v at a greater value implies that a sale on day d for id would be a loss


In [ ]:
import math

def prelec(p, gamma):

    #print("prelec, entry",p)
    x=math.exp(-(-math.log(p)) ** gamma)

    return x

In [ ]:
def PTv1(price, sold, stored, alpha, beta, lam, gam) : #i_d, n, N
  gain = sum((prelec(p[j],gam) * (sold*(price-j))**alpha) for j in range(1, price))
  loss= sum((prelec(p[j],gam) * lam*(sold*(price-j))**beta ) for j in range(price+1, I+1))
  return gain-loss


In [ ]:
PTv1(6,1,1,1,1,1.755,1)

3.93575

In [ ]:
def h_probPT(day_f, actual_day,gamma, cutoff): # This returns the probability of a sale on the day (d=h) where day is the index variable used in the util represented by f and actual day is the actual day used in the util represented by "day"

  h_rem= 1
  for h in range(actual_day-1, day_f+1, -1):
    h_rem*=sum(prelec(p[k],gamma) for k in range(1,cutoff[h]-1))

  #print("h_Rem for day", actual_day, "is ",h_rem )
  return h_rem

In [ ]:
def PTv2(day, price, sold, alpha, beta, lam, gam, c_o ):
  # this is our code and it is copied straight from what's shown on the report. PTv3 is a "working" version however it doesn't adhere directly to the model in the report
  gain=sum(  (prelec(  p[j]+ sum(h_probPTv2(day-1,f,c_o,gam) for f in range(day-2,1,-1)) * p[j], gam ) * (sold*(price-j))**alpha )   for j in range(c_o[day-1],price))

 # print("gain for day", day,"is ", gain)
  loss=sum((prelec(  p[j]+ sum(h_probPTv2(day-1,f,c_o,gam) for f in range(day-2,1,-1)) * p[j] , gam) * lam*(sold*(j-price))**beta )   for j in range(max(c_o[day-1],price+1),I+1))
  #print("loss for day", day,"is ", loss)
  util=gain-loss
 # print("util for day", day, "is", util)
  return util

In [ ]:

def h_probPTv2(day, f, c_o,gam):
  prob=1.0
  for h in range(1,f+1):
    prob*=sum(prelec(p[j],gam) for j in range(1,c_o[day-h]))
  return prob

In [ ]:
def obj_gain(day, price, sold, alpha, beta, lam, gam, c_o,j):
  gain=sold*(price-j)
  gain=gain**alpha
  gain*=prelec(p[j],gam)
  return gain


In [ ]:
def obj_loss(day, price, sold, alpha, beta, lam, gam, c_o,j):
  loss=sold*(j-price)
  loss=loss**beta
  loss*=lam*prelec(p[j],gam)
  return loss

In [ ]:

def PTv3(day,price,sold,alpha,beta,lam,gam,c_o):
  util=0
  for j in range(c_o[day-1],price):
    util+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for j in range(max(price+1,c_o[day-1]), I+1):

    util-=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)


  for k in range(1,day+2):

    h_prob=h_probPTv2(day,k-1,c_o,gam)
    #h_prob=h_probPT(day_f=k, actual_day=day,gamma=gam,cutoff=c_o)

    gain_prob=0
    for j in range(c_o[day-k],price):
      gain_prob+=obj_gain(day,price,sold,alpha,beta,lam,gam,c_o,j)

    loss_prob=0
    for j in range(max(price+1,c_o[day-k]), I+1):
      loss_prob+=obj_loss(day,price,sold,alpha,beta,lam,gam,c_o,j)

    util+=h_prob*(gain_prob-loss_prob)




  #print("util for day",day,"is ", util)
 # print("gain: "  ,gain_prob)
 # print("loss: ", loss_prob)

  return util





In [ ]:
testlist = [1]*68

In [ ]:
PTv3(7,8,4,1,1,1.755,1,testlist)

6.046399999999999

to compute cutoff prices we iterate through each price (1-15 )and determine if they will result in a gain or loss. I


In [ ]:
def max_units(day,price, stored,a,b,l,g,cut_offs):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PTv3(day,price,units,a,b,l,g,cut_offs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [ ]:
DAYS

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67]

In [ ]:
def info_return(day, participant):
  curr_df= df_days[day].iloc[participant] # data acquiring is fine.
  n_1=curr_df["Sold"+str(day+1)]
  #print("Sold units on day :",day, " is : ", n_1)

  N_d=curr_df["Stored"+str(day+1)]
  #print("Stored on day",day,"is:", N_d)
  i_d=curr_df["Price"+str(day+1)]
 # print("Price on day",day,"is",i_d)
  return(n_1,N_d,i_d)

In [ ]:
def cutoff_pt_list(a,b,l,g):#this function generates a list before we make our predicitons
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0:
      continue
    price=cutoffs[day-1]
    prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)
    counts=0
    while (prosp <= 0 and price < 15):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                prosp = PTv3(day,price,1,a,b,l,g,cutoffs)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
   # print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs





In [ ]:
def fitPTv2(participant,a,b,l,g):
  predictions=[]                 #[0]*68
  #cut_offs=c_o
  cut_offs=cutoff_pt_list(a,b,l,g)            #cutoff_PT(a,b,l,g,participant)


  for day in DAYS: # 1,2,3
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]

    ''' c_o=cutoff_PT(a,b,l,g,day,cut_offs,stored)
    print("cut off price for day", day,"is ", c_o)
    cut_offs[day-1]=c_o'''
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units(day,price,stored,a,b,l,g, cut_offs)
      else:
        print(".")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  print("DATA ADDED :)")
  return predictions,cut_offs

In [ ]:
'''def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  if tw>(68-day):
    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)
    print("util PT : ", util)

  else:
    gain=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(j+1,I+1)))**k for k in range(1,tw-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(1,price))
    loss=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(price+2,I+1)))**k for k in range(1,tw-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(price+1,I+1))
    util=gain-loss
    print("util", util)

  return util
'''





'def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):\n  if tw>(68-day):\n    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)\n    print("util PT : ", util)\n\n  else:\n    gain=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(j+1,I+1)))**k for k in range(1,tw-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(1,price))\n    loss=sum(prelec(p[j] + sum((1-sum(p[i] for i in range(price+2,I+1)))**k for k in range(1,tw-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(price+1,I+1))\n    util=gain-loss\n    print("util", util)\n\n  return util\n'

In [ ]:
def PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  t=int(tw)

  if t>day: # 68,67,66
    util=PTv3(day,price,sold,alpha,beta,lam,gam,c_o)
    #print("util PT : ", util)
  else:
    gain=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j], gam)* (sold*(price-j))**alpha for j in range(c_o[t-1],price))
    loss=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j],gam)*lam*(sold*(j-price))**beta for j in range(max(c_o[t-1],price+1),I+1))
    util=gain-loss
    #print("util day:" ,day,util)
  return util

In [ ]:
def PT_TW_c(day,price,sold,alpha,beta,lam,gam,tw,c_o):
  util_sum=0
  for day in range(day,0,-1):
    util_sum+=PT_TW(day,price,sold,alpha,beta,lam,gam,tw,c_o)
  return util_sum

In [ ]:
def cutoff_pt_tw_list(a,b,l,g,tw): # 6.11.24 changed
  t=tw
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]
    ''' if day<t:
          prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
    else:
          prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)
    print(prosp)'''
    prosp=PT_TW_c(day,price,1,a,b,l,g,t,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                '''if day<t:
                  prosp=PTv3(day,price,1,a,b,l,g,cutoffs)
                else:
                  prosp = PT_TW(day,price,1,a,b,l,g,t,cutoffs)'''
                prosp=PT_TW_c(day,price,1,a,b,l,g,t,cutoffs)
                #print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    #print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [ ]:
#print(load_fit('/content/drive/MyDrive/Glass Lab Data/PT fits/DIRECT fits/participant 17 September 02, 2023.pickle'))

In [ ]:
#count_errorPT_TW([0.72222222, 0.83333333, 1.16666667, 0.16666667,68],16)

In [ ]:
'''def cutoff_pt_tw_list(a,b,l,g,tw):
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]

    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1

                prosp = PT_TW(day,price,1,a,b,l,g,tw,cutoffs)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs'''

'def cutoff_pt_tw_list(a,b,l,g,tw):\n  cutoffs=[1]\n  for day in DAYS:#0,1,2,3\n    if day==0: # last day\n      continue\n    price=cutoffs[day-1]\n\n    print(prosp)\n    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)\n\n    counts=0\n    while (prosp <= 0 and price < I):\n                # Continually increment price until it is worthwhile to sell\n                price += 1\n                counts+=1\n\n                prosp = PT_TW(day,price,1,a,b,l,g,tw,cutoffs)\n                print("prosp on day", day, "is ", prosp)\n                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)\n\n\n\n    cutoffs.append(price)\n    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)\n  return cutoffs'

In [ ]:
def max_units_PTTW(day,price,stored,a,b,l,g,tw,cutoffs ):
  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PT_TW_c(day,price,units,a,b,l,g,tw,cutoffs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  return pred

In [ ]:
def fitPT_TW(participant,a,b,l,g,tw):
  predictions=[]                 #[0]*68
  cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)
  #cut_offs=cutoff_pt_tw_list(a,b,l,g,tw)            #cutoff_PT(a,b,l,g,participant)

  rev_d=reversed(DAYS)
  #rev_d=DAYS
  for day in rev_d: # 67,66,65...
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units_PTTW(day,price,stored,a,b,l,g,tw,cut_offs)



    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  #print("DATA ADDED :)")
  return predictions,cut_offs

In [ ]:
z=fitPT_TW(7,1,1,	1.3	,.1,2)

In [ ]:
print(z)

([0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 4], [1, 9, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])


In [ ]:
sum(z[0])

66

In [ ]:
def h_probPT_DD(day, f, c_o,gam):
  prob=1.0
  for h in range(day-1,f+2): # day is added 1  here
    prob*=sum(p[k] for k in range(1,c_o[day-1])) # I don't think this will work, replacement is c_o[day-h] as that is what Yudelson did TALK tO GLASS # Trying c_o[day-1]
  return prob

In [ ]:
def obj_gain_DD(day, price, sold, alpha, beta, lam, gam, c_o,j, k):
  gain=sold*(price-j)/(1+k*(day-1))
  gain=gain**alpha
  gain*=prelec(p[j],gam)
  return gain


In [ ]:
def obj_loss_DD(day, price, sold, alpha, beta, lam, gam, c_o,j, k):
  loss=sold*(j-price)/(1+k*(day-1))
  loss=loss**beta
  loss*=lam*prelec(p[j],gam)
  return loss

In [ ]:
def PT_DD(day,price,sold,alpha,beta,lam,gam,c_o, k):
  # since day starts from 0, 1, 2,3,4 for the list DAYS but the equation takes d=1 as the last day so all days in this equation are added 1
  lb_1=c_o[day-1] # this stays the same as it is just the last days cutoff value
  ub_1=price
  print("ub_1-lb_1: ", ub_1-lb_1)
  gain= sum(
      (
          prelec(p[j],gam)*(sold*(price-j)/(1+k*(day-1)))**alpha # day is added 1 here

       )
      for j in range(lb_1, ub_1)
      )
  lb_2=max(lb_1,price+1)
  ub_2=I+1
  print("ub_2-lb_2: ", ub_2-lb_2)
  loss= sum(

            (
              prelec(
                  (p[j] + sum(h_probPT_DD(day,f,c_o,gam) for f in range(day-2,0,-1))*p[j] # day is added 1 here edit. not anymore
               ),gam)


            )
            * lam*(sold*(j-price)/(1+k*(day-1)))**beta # day is added 1 here
            for j in range(lb_2,ub_2)

  )

  util=gain-loss
  print("g:", gain)
  print("l", loss)
  return util


In [ ]:
def PT_DD_c(day,price,sold,alpha,beta,lam,gam,c_o, k):
  total_util=0
  for d in range(day,0,-1):
    print("day",d)
    total_util+=PT_DD(d,price,sold,alpha,beta,lam,gam,c_o, k)
  return total_util

In [ ]:
def PT_DD_v2(day,price,sold,alpha,beta,lam,gam,c_o, kay):
  util=0
  for j in range(c_o[day-1],price):
    util+=obj_gain_DD(day,price,sold,alpha,beta,lam,gam,c_o,j,kay)


  for j in range(max(price+1,c_o[day-1]), I+1):

    util-=obj_loss_DD(day,price,sold,alpha,beta,lam,gam,c_o,j,kay)


  for k in range(1,day+2):

    h_prob=h_probPT_DD(day,k-1,c_o,gam)
    #h_prob=h_probPT(day_f=k, actual_day=day,gamma=gam,cutoff=c_o)

    gain_prob=0
    for j in range(c_o[day-k],price):
      gain_prob+=obj_gain_DD(day,price,sold,alpha,beta,lam,gam,c_o,j,kay)

    loss_prob=0
    for j in range(max(price+1,c_o[day-k]), I+1):
      loss_prob+=obj_loss_DD(day,price,sold,alpha,beta,lam,gam,c_o,j,kay)

    util+=h_prob*(gain_prob-loss_prob)
    # util += gain_prob - (h_prob * loss_prob)


  # # since day starts from 0, 1, 2,3,4 for the list DAYS but the equation takes d=1 as the last day so all days in this equation are added 1
  # lb_1=c_o[day-1] # this stays the same as it is just the last days cutoff value
  # ub_1=price
  # print("ub_1-lb_1: ", ub_1-lb_1)


#   gain= sum(
#       (
#           prelec(p[j],gam)*(sold*(price-j)/(1+k*(day-1)))**alpha # day is added 1 here

#        )
#       for j in range(lb_1, ub_1)
#       )
#   lb_2=max(lb_1,price+1)
#   ub_2=I+1
#   print("ub_2-lb_2: ", ub_2-lb_2)
#   loss= sum(

#             (
#               prelec(
#                   (p[j] + sum(h_probPT_DD(day,f,c_o,gam) for f in range(day-2,0,-1))*p[j] # day is added 1 here edit. not anymore
#                ),gam)


#             )
#             * lam*(sold*(j-price)/(1+k*(day-1)))**beta # day is added 1 here
#             for j in range(lb_2,ub_2)

#   )


  # util=gain-loss
  # print("g:", gain_prob)
  # print("l", loss_prob)
  print("util:", util)
  return util


In [ ]:
def cutoff_PT_DD_list(a,b,l,g,k):

  cutoffs=[1] #last day (d=1) but we used day=0
  for day in DAYS:#0,1,2,3 but actual days start 1, 2,3,4
    if day==0: # last day
      continue
    price=cutoffs[day-1] # last day, also the same as the lower bound for the gain in PT-DD


    prosp = PT_DD_c(day,price,1,a,b,l,g,cutoffs,k) # sold is one as long as 1 will sell its fine
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1

                prosp = PT_DD_c(day,price,1,a,b,l,g,cutoffs,k)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [ ]:
x=cutoff_PT_DD_list(.5,.5,1.5,.9,0)

Streaming output truncated to the last 5000 lines.
l 0
day 41
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 40
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 39
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 38
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 37
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 36
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 35
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 34
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 33
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 32
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 31
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 30
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 29
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 28
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826
l 0
day 27
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826
l 0
day 26
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826
l 0
day 25
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826
l 0
day 24
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826
l 0
day 23
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.032935760967237826

In [ ]:
def max_units_PT_DD(day,price, stored,a,b,l,g,k,cut_offs):

  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PT_DD_c(day,price,units,a,b,l,g,cut_offs,k)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  print("predicted_unit:", pred)
  return pred

In [ ]:
def fit_PT_DD(participant,a,b,l,g,k):

  predictions=[]                 #[0]*68
  #cut_offs=c_o
  cut_offs=cutoff_PT_DD_list(a,b,l,g,k)            #cutoff_PT(a,b,l,g,participant)


  for day in DAYS: # 1,2,3
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]

    ''' c_o=cutoff_PT(a,b,l,g,day,cut_offs,stored)
    print("cut off price for day", day,"is ", c_o)
    cut_offs[day-1]=c_o'''
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units_PT_DD(day,price,stored,a,b,l,g, k,cut_offs)
      else:
        print(".")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  #predictions_df['Participant '+str(participant+1)]=pred_series.values
 # predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  print("DATA ADDED :)")
  return predictions,cut_offs


In [ ]:
x=fit_PT_DD(0,0.59200,0.62962,1.09650,0.98311,0)

Streaming output truncated to the last 5000 lines.
g: 3.5796788560182664
l 0
day 3
ub_1-lb_1:  7
ub_2-lb_2:  0
g: 5.036953378414555
l 0
day 2
ub_1-lb_1:  8
ub_2-lb_2:  0
g: 6.804619056884204
l 0
day 1
ub_1-lb_1:  14
ub_2-lb_2:  0
g: 19.716544524148716
l 0
day 26
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 25
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 24
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 23
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 22
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 21
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 20
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 19
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.12495310219926938
l 0
day 18
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.4031066580949352
l 0
day 17
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.4031066580949352
l 0
day 16
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.4031066580949352
l 0
day 15
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.4031066580949352
l 0
day 

In [ ]:
x[0]

[4,
 0,
 0,
 28,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 56,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
def h_probPT_DD_TW(day, f, c_o,gam):

    c=-1 if day<=2 else 1

    product=1
    for h in range(day-1, f+2,c):
      product*=sum(p[k] for k in range (1,c_o[h-1]))

    return product



if d<2

range for f has to increase
so c=1



if d=2

In [ ]:
day=2
c=1 if day<3 else -1
c_o=[1,8]
if day==2:
  c_ub=1
else:
  c_ub=0
loss=sum (prelec( sum(h_probPT_DD_TW(day,f,c_o,.5) for f in range(day-2,c_ub,c)) * p [j] , .5) * 1.5*(1*(j-1)/(1+0*(day-1)))**.5 for j in range(1+1, I+1))

In [ ]:
loss

9.963774485639407

In [ ]:
prod=1
for h in range(4,6):
   prod*=sum(p[k] for k in range (1,11))

In [ ]:
prod

0.7225000000000001

In [ ]:

def PT_TW_DD(day,price,sold,alpha,beta,lam,gam,tw, K , s, F, c_o):

  gain=0
  loss=0
  g2=0
  l1=0
  boo=day<=2
  c=1 if boo else -1

  if day==2:
    c_ub=1
  else:
    c_ub=0
# why is Delay Discounting formula different ? im leaving it as minus as shown until Glass gets back to me, but I will continue to have it be
  if tw<day and day<68:
    lb=c_o[day-1]
    ub=price

    gain= sum((prelec(p[j]+ sum((1-sum( p[h] for h in range (j+1,I+1)))**k for k in range(1,tw-1)) *p[j]










                     ,gam) * (sold*(price-j)/(1+K*(day-1)))**alpha )

              for j in range(lb,ub))



    lb1=max(c_o[day-1],price+1)
    ub=I+1
    loss= sum((prelec(p[j]+ sum((1-sum( p[h] for h in range (j+1,I+1)))**k for k in range(1,tw-1)) *p[j]













                     ,gam) * lam*(sold*(j-price)/(1+K*(day-1)))**beta )

              for j in range(lb,ub))

    g2=0
    l1=0

  elif tw>day:


    lb=c_o[day-1]
    ub=price

    gain=sum((prelec(p[j], gam) *(sold*(price-j)/(1+K*(day-1)))**alpha)  for j in range(lb,ub) )

    lb2 =max(c_o[day-1],price+1)
    ub2=I+1
    l1=sum((prelec(p[j], gam)* lam*(sold*(j-price)/((1+K*(day-1))**s))**beta)  for j in range(lb2,ub2) ) # added lambda in this component

    g2=sum (prelec( sum(h_probPT_DD_TW(day,f,c_o,gam) for f in range(day-2,c_ub,c)) * p [j] , gam) * (sold*(price-j)/(1+K*(day-F)))**alpha for j in range(c_o[day-1], price))


    loss=sum (prelec( sum(h_probPT_DD_TW(day,f,c_o,gam) for f in range(day-2,c_ub,c)) * p [j] , gam) * lam*(sold*(j-price)/(1+K*(day-F)))**beta for j in range(price+1, I+1))

  util =  gain - loss +g2 - l1
  print("gain", gain, "loss", loss, "g2",g2, "l1", l1)

  print("util")

  return util




In [ ]:

def PT_TW_DD_v3(day,price,sold,alpha,beta,lam,gam,tw, K , s, F, c_o):

  gain=0
  loss=0
  g2=0
  l1=0
  boo=day<=2
  c=1 if boo else -1

  if day==2:
    c_ub=1
  else:
    c_ub=0
# why is Delay Discounting formula different ? im leaving it as minus as shown until Glass gets back to me, but I will continue to have it be
  if tw<day and day<68:
    lb=c_o[day-1]
    ub=price

    gain= sum((prelec(p[j]+ sum((1-sum( p[h] for h in range (j+1,I+1)))**k for k in range(1,tw-1)) *p[j]










                     ,gam) * (sold*(price-j)/(1+K*(day-1)))**alpha )

              for j in range(lb,ub))



    lb1=max(c_o[day-1],price+1)
    ub=I+1
    loss= sum((prelec(p[j]+ sum((1-sum( p[h] for h in range (j+1,I+1)))**k for k in range(1,tw-1)) *p[j]













                     ,gam) * lam*(sold*(j-price)/(1+K*(day-1)))**beta )

              for j in range(lb,ub))

    g2=0
    l1=0

  elif tw>day:

    '''
    lb=c_o[day-1]
    ub=price

    gain=sum((prelec(p[j], gam) *(sold*(price-j)/(1+K*(day-1)))**alpha)  for j in range(lb,ub) )

    lb2 =max(c_o[day-1],price+1)
    ub2=I+1
    l1=sum((prelec(p[j], gam)* lam*(sold*(j-price)/((1+K*(day-1))**s))**beta)  for j in range(lb2,ub2) ) # added lambda in this component

    g2=sum (prelec( sum(h_probPT_DD_TW(day,f,c_o,gam) for f in range(day-2,c_ub,c)) * p [j] , gam) * (sold*(price-j)/(1+K*(day-F)))**alpha for j in range(c_o[day-1], price))


    loss=sum (prelec( sum(h_probPT_DD_TW(day,f,c_o,gam) for f in range(day-2,c_ub,c)) * p [j] , gam) * lam*(sold*(j-price)/(1+K*(day-F)))**beta for j in range(price+1, I+1))

  util =  gain - loss +g2 - l1
  print("gain", gain, "loss", loss, "g2",g2, "l1", l1)

  print("util")
  '''
  util=PT_DD(day,price,sold,alpha,beta,lam,gam,c_o, K)
  print("util PTDD : ", util)


  return util




In [ ]:
def PT_TW_DD_c(day,price,sold,alpha,beta,lam,gam,tw, K , s, F, c_o):
  total_util=0
  for d in range(day,0,-1):
    print("day",d)
    total_util+=PT_TW_DD_v3(d,price,sold,alpha,beta,lam,gam,tw, K , s, F, c_o)
  return total_util

In [ ]:
gain=sum((prelec(p[j], .5) *(1*(2-j)/(1+0*(1-1)))**.5)  for j in range(1,2) )



l1=sum((prelec(p[j], .5) *(1*(j-1)/((1+0*(1-1))**1))**.5)  for j in range(2,16) )

gain
print("l1",l1)

g2=sum(prelec( sum(h_probPT_DD_TW(day,f,c_o,5) for f in range(day-2,c_ub,c)) * p[j] , .5) * (1*(10-j)/(1+0*(day-1)))**.5 for j in range(8, 10))
loss=sum(prelec( sum(h_probPT_DD_TW(day,f,c_o,.5) for f in range(day-2,c_ub,c)) * p [j] , .5) * 1.5*(1*(j-10)/(1+0*(day-1)))**.5 for j in range(10, I+1))

print("g2", g2*5, "loss", loss)

l1 6.642516323759605
g2 2.4219868550487247 loss 1.8251853261916366


In [ ]:
prelec(.12,.5)

0.2331408564982428

In [ ]:
l1

6.642516323759605

In [ ]:
h_probPT_DD_TW(14,1,x,.8)

1

In [ ]:
sum(f for f in range(-2,0,-1))

0

In [ ]:
def PT_TW_DD_v2(day,price,sold,alpha,beta,lam,gam,tw, K , s, F, c_o):

  t=int(tw)

  if t>day: # 68,67,66
    util=PT_DD_v2(day,price,sold,alpha,beta,lam,gam,c_o, K)
    print("util PT : ", util)
  else:
    gain=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j], gam)* (sold*(price-j)/(1+K*(day-1)))**alpha for j in range(c_o[t-1],price))
    loss=sum(prelec(p[j] + sum((sum(p[k] for k in range(1,c_o[t-1])))**h for h in range(1,t-1)) * p[j],gam)*lam*(sold*(j-price)/(1+K*(day-1)))**beta for j in range(max(c_o[t-1],price+1),I+1))
    util=gain-loss
    print("util day:" ,day,util)
  return util

In [ ]:
def cutoff_pt_tw_dd_list(a,b,l,g,k,f,s,tw):
  t=tw
  cutoffs=[1]
  for day in DAYS:#0,1,2,3
    if day==0: # last day
      continue
    price=cutoffs[day-1]
    prosp=PT_TW_DD_c(day,price,1,a,b,l,g,tw,k,s,f,cutoffs)
    print(prosp)
    #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)

    counts=0
    while (prosp <= 0 and price < I):
                # Continually increment price until it is worthwhile to sell
                price += 1
                counts+=1
                prosp=PT_TW_DD_c(day,price,1,a,b,l,g,tw,k,s,f,cutoffs)
                print("prosp on day", day, "is ", prosp)
                #prosp=PTv2(day,price,1,a,b,l,g,cutoffs)



    cutoffs.append(price)
    print("counts from the previous value of", cutoffs[day-1], " is ", counts,"cutoff price is now", price, "for day ",day)
  return cutoffs

In [ ]:
def max_units_PT_TW_DD(day,price, stored,a,b,l,g,tw,k,s,f, cut_offs):

  sells=[]
  pred=0
  greatest=-100

  for units in range(1,stored+1):

    prosp=PT_TW_DD_c(day,price,units,a,b,l,g,tw,k,s,f,cut_offs)

    if prosp>greatest:
      greatest=prosp
      pred=units
      sells.append(units)

#  print("all possibile units to sell at", sells)

 # print("len of possible sells:", len(sells))

    #if prosp>greatest:
       # greatest=prosp
       # pred=units
  print("predicted_unit:", pred)
  return pred

In [ ]:
predictions_df=pd.DataFrame()

In [ ]:
def fit_PT_TW_DD(participant,a,b,l,g,k,f,s,tw):

  predictions=[]                 #[0]*68
  #cut_offs=c_o
  cut_offs=cutoff_pt_tw_dd_list(a,b,l,g,k,f,s,tw)            #cutoff_PT(a,b,l,g,participant)


  for day in DAYS[::-1]: # 67,66
    pred=0
    vals=info_return(day, participant) # sold, stored,price
    stored=vals[1]
    price=vals[2]

    ''' c_o=cutoff_PT(a,b,l,g,day,cut_offs,stored)
    print("cut off price for day", day,"is ", c_o)
    cut_offs[day-1]=c_o'''
    if day==0:
      pred=stored
    else:

      if price>=cut_offs[day]:

        pred=max_units_PT_TW_DD(day,price,stored,a,b,l,g,tw, k, s,f ,cut_offs)
      else:
        print(".")

    predictions.append(pred)
    #predictions.insert(day-1,pred)
  pred_series=pd.Series(predictions)
  predictions_df['Participant '+str(participant+1)]=pred_series.values
  predictions_df['Participant' +str(participant+1)+' cutoffs']=pd.Series(cut_offs).values
  #print("DATA ADDED :)")
  return predictions,cut_offs


In [ ]:
x=fit_PT_TW_DD(0,0.59200,0.62962,1.09650,0.98311,0,0,0,68)



Streaming output truncated to the last 5000 lines.
day 17
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 16
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 15
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 14
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 13
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 12
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.5071254361510678
l 0
util PTDD :  0.5071254361510678
day 11
ub_1-lb_1:  3
ub_2-lb_2:  0
g: 1.094026465350847
l 0
util PTDD :  1.094026465350847
day 10
ub_1-lb_1:  3
ub_2-lb_2:  0
g: 1.094026465350847
l 0
util PTDD :  1.094026465350847
day 9
ub_1-lb_1:  3
ub_2-lb_2:  0
g: 1.094026465350847
l 0
util PTDD :  1.094026465350847
day 8
ub_1-lb_1:  3
ub_2-lb_2:  0
g: 1.094026465350847
l 0
util PTDD :  1.094026465350847
day 7
ub_1-lb_1:  4
ub_2-lb_2

In [ ]:
x=cutoff_pt_tw_dd_list(.592,0.629,1.096,.983,0,0,1,65)

Streaming output truncated to the last 5000 lines.
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 35
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 34
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 33
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 32
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 31
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 30
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 29
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 28
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413
day 27
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.021876307029770413
l 0
util PTDD :  0.021876307029770413


In [ ]:
cutoff_PT_DD_list(.5,.5,1.5,.5,0)

Streaming output truncated to the last 5000 lines.
l 0
day 41
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 40
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 39
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 38
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 37
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 36
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 35
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 34
ub_1-lb_1:  0
ub_2-lb_2:  0
g: 0
l 0
day 33
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 32
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 31
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 30
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 29
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 28
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 27
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 26
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 25
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.13836177500750083
l 0
day 24
ub_1-lb_1:  1
ub_2-lb_2:  0


[1,
 9,
 10,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15]

In [ ]:
print(list(range(15,3,-1)))

[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4]


In [ ]:
parameters_df=pd.read_csv('/content/drive/MyDrive/Glass Lab Data/PT Parameter Spreadsheet - Sheet1 (1).csv')

In [ ]:
parameters_df

,Participant #,Alpha,Beta,Lambda,Gamma,MDev Error
0,1,0.59200,0.62962,1.09650,0.98311,0.000
1,2,0.44107,0.47101,1.18092,0.95782,0.015
2,3,0.88028,0.96896,1.01945,0.96526,0.044
3,4,0.94566,0.96252,1.56756,0.94092,0.134
4,5,0.88363,0.98226,1.07153,0.91291,0.059
5,6,0.83333,0.53704,1.61111,0.72222,0.000
6,7,0.82733,0.90898,1.18740,0.82933,0.132
7,8,0.87629,0.87952,1.61390,0.05490,0.000
8,9,0.83226,0.87224,1.02140,0.98157,0.412
9,10,0.31544,0.31963,1.27594,0.51131,0.000


In [ ]:
def PREDICT_YES():
  for dude in range(0,56):
    a=parameters_df._get_value(dude,col='Alpha')
    b=parameters_df._get_value(dude,col='Beta')
    l=parameters_df._get_value(dude,col='Lambda')
    g=parameters_df._get_value(dude,col='Gamma')

    fit_PT_TW_DD(dude,a,b,l,g,0,0,0,68)

In [ ]:
predictions_df

,Participant 1,Participant1 cutoffs
0,0,1
1,0,7
2,0,8
3,0,9
4,0,10
...,...,...
63,0,15
64,28,15
65,0,15
66,0,15


In [ ]:
soldOnly = SellData
for name in SellData.columns :
  if 'Sold' not in name:
    soldOnly=soldOnly.drop(name, axis=1)

soldOnly = soldOnly.T
soldOnly = soldOnly.iloc[::-1]
soldOnly

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
Sold1,4,3,3,4,4,4,4,4,4,4,...,3,4,3,10,6,3,2,12,2,2
Sold2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
Sold3,0,1,1,0,0,0,0,0,0,0,...,1,1,1,0,0,1,1,0,2,1
Sold4,28,26,12,12,12,12,7,12,6,12,...,5,4,3,20,6,2,1,5,3,1
Sold5,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sold64,0,0,0,0,0,0,0,0,0,0,...,0,1,1,4,1,1,0,1,0,2
Sold65,0,5,2,5,5,7,3,7,0,7,...,3,3,2,4,1,7,5,0,3,4
Sold66,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,1
Sold67,0,0,0,0,0,0,0,0,0,0,...,0,1,3,0,1,0,2,1,1,0


In [ ]:
list(soldOnly[0].values)

[4,
 0,
 0,
 28,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 56,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
import random

In [ ]:
from scipy.optimize import minimize

In [ ]:
from scipy.optimize import Bounds

In [ ]:
from sklearn.metrics import d2_absolute_error_score as d2a

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def avg_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)
    stored=vals[1]
    sold=vals[0]
    if stored==0:
      d0+=1
      continue
    else:
      pred=preds[day]
      #pred=predictionsOnly._get_value(day,"Participant"+str(participant))
      day_err=(pred-sold)/stored
      total+=day_err
  return total/(68.0-d0)


In [ ]:
def get_loss_one(params,participant):#a,b,l,g
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return mse


In [ ]:
def d2a_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]


  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  mse=mean_squared_error(actuals,preds)
  error=d2a(actuals,preds)

  return 1-error


In [ ]:
def fit_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]



  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)
  actuals=actuals[::-1]

  error=d2a(actuals,preds)

  return 1-error

In [ ]:
def fit_errorPTTW_dir(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)


  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  actuals=list(soldOnly[participant].values)

  error=d2a(actuals,preds)

  return error

# THE MAIN ERROR FUNCTION WE ARE USING FOR OPTIMIZATION


In [ ]:
def count_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]

  vals=fitPTv2(participant,a,b,l,g)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[day]
    day_err=abs(sold-pred)
    total+=day_err
  return total

In [ ]:
def count_errorPT_TW(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  tw=int(tw)

  vals=fitPT_TW(participant,a,b,l,g,tw)
  preds=vals[0]
  total=0
  d0=0
  for day in DAYS[::-1]:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[67-day]
    day_err=abs(sold-pred)
    total+=day_err
  return total


In [ ]:
fit_errorPT_TW([1,1,	1.3	,.1,2],7)

0.2666666666666667

In [ ]:

def count_error_PT_DD(params, participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  k=params[4]
  preds=fit_PT_DD(participant, a,b,l,g,k)[0]
  total=0
  for day in DAYS:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[day]
    day_err=abs(sold-pred)
    total+=day_err
  return total



In [ ]:
def display_errors(four_fits, start,stop):
  avg_errors=[]
  d2a_errors=[]
  mse_errors=[]
  count_errors=[]
  iter=list(zip(four_fits, list(range(start,stop))))
  for fit,participant in iter:
    x=avg_error(fit,participant)
    y=d2a_error(fit,participant)
    z=get_loss_one(fit,participant)
    m=count_error(fit,participant)
    avg_errors.append(x)
    d2a_errors.append(y)
    mse_errors.append(z)
    count_errors.append(m)
  return avg_errors,d2a_errors,mse_errors,count_errors

In [ ]:
bound_set=set(list(range(1,69)))

In [ ]:
losstest=fit_errorPT_TW(np.array([1,1,1.5,1,68]),0)

In [ ]:
losstest

0.6363636363636364

In [ ]:
def default_cb(xk):
  print(xk)
  ret=False
  if count_error(xk,14)<=11:
    ret=True
  return ret

In [ ]:
#GUESS=np.array([.5,.5,1,.5,30])
GUESS=[.5,.5,1,.5,30]

In [ ]:
from scipy.optimize import differential_evolution
from scipy.optimize import LinearConstraint

In [ ]:
def bold_text(text):
    bold_start = '\033[1m'
    bold_end = '\033[0m'
    return bold_start + text + bold_end


print('website: ' + bold_text('bobbyhadz.com') + ' abc 123')


website: bobbyhadz.com abc 123


In [ ]:
count_error_PT_DD([.441,.471,1.18, .957, 0], 1)

Streaming output truncated to the last 5000 lines.
g: 1.7534941811234188
l 0
day 3
ub_1-lb_1:  7
ub_2-lb_2:  0
g: 2.3989549852122725
l 0
day 2
ub_1-lb_1:  8
ub_2-lb_2:  0
g: 3.161970293338316
l 0
day 1
ub_1-lb_1:  14
ub_2-lb_2:  0
g: 8.46353140982402
l 0
day 26
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 25
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 24
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 23
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 22
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 21
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 20
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 19
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 18
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 17
ub_1-lb_1:  1
ub_2-lb_2:  0
g: 0.08003099926445681
l 0
day 16
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.23681412631074059
l 0
day 15
ub_1-lb_1:  2
ub_2-lb_2:  0
g: 0.23681412631074059
l 0
d

8

In [ ]:
cb_index=0

In [ ]:
our_fits_df=pd.DataFrame()

In [ ]:
from time import time
import warnings

In [ ]:
def dict_constraint(xk):
  return xk[1]-xk[0]

In [ ]:
check=lambda params: params if params[0]<=params[1] else "Alpha greater than Beta"

In [ ]:

def differential_evolution_participant(participant,save):
    part=participant
    #init_=init_vertices()
   #init_=init_[0]
    #init_=np.array(init_)
    g=np.array([.5,.5,1,.5,60])
    #g=guess
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(7,68)] # changing the bounds might be the reason why it's messing up, but a and b cannot be greater than 1 keep in mind
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    result = differential_evolution(func=fit_errorPT_TW,x0=g,bounds=bounds, args=(part,), strategy="best1bin", maxiter=30000, tol=0.0002, disp=True,updating="immediate",constraints=de_lc,polish=True,integrality=[False,False,False,False,True] ) # can change back to get_loss_one or avg error if needed
    best_fit = result.x
    print(result.message)
    if save:
      save_fit(best_fit,participant+1,"diffev")
    return best_fit


In [ ]:
from scipy.optimize import direct
def direct_participant(participant,dum):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    g=np.array([.5,.5,1,.5,68])
    result = direct(func=fit_errorPT_TW, bounds=bounds, args=(part,), maxiter=10000, locally_biased=False)

    best_fit = result.x
    print (result.message)
    if dum:
      save_fit(check(best_fit),participant+1,'direct')
    return best_fit

In [ ]:
from scipy.optimize import shgo
def shgo_participant(participant):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    dc=({'type': 'ineq','fun':dict_constraint})
    g=np.array([.5,.5,1,.5,50])
    result = shgo(func=fit_errorPT_TW, bounds=bounds, args=(part,) , options={'maxiter':10000},constraints=dc)

    best_fit = result.x
    print (result.message)

    return best_fit

In [ ]:
from scipy.optimize import minimize
def nelder_mead_participant(participant, guess):
  part = participant
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), bounds=bounds, method='Nelder-Mead', options={"fatol":.15})
  return result.x

cob_eq=lambda x: x[1]-x[0]

def cobyla_participant(participant,guess):
  part = participant
  bounds = [(0, 1), (0, 1), (1, 2), (0, 1), (2,68)]
  g=guess
  result=minimize(fit_errorPT_TW,x0=g,args=(participant,), method='COBYLA', bounds=bounds, constraints={"type":"ineq", "fun":cob_eq}, options={"tol":.13})
  return result.x

In [ ]:
from scipy.optimize import basinhopping
def basin_hopping_participant(participant):
    part = participant
    bounds = [(0, 1), (0, 1), (1, 2), (0, 1),(2,68)]

    #this is the basic function plus maxiter because that's the barebones for what basically the rest of them have but I can add more parameteres if this doesn't work
    de_lc=LinearConstraint(A=[1,-1,0,0,0],lb= -np.inf ,ub=0)
    g=np.array([.5,.5,1,.5,50])
    opt_func=lambda params: fit_errorPT_TW(params,participant)
    result = basinhopping(func=opt_func, minimizer_kwargs={"method":"Nelder-Mead"}, x0=g, niter=10000, interval=75,T=.5)

    best_fit = result.x
    print (result.message)

    return best_fit

In [ ]:
def global_polish(participant,starter,count,glazer):
  silly=None
  s=''
  if starter=='diffev':
    silly=differential_evolution_participant
    s='neighbors'
  elif starter=='direct':
    silly= direct_participant
    s='polished_direct'


  ret='didnt work'
  start=silly(participant,False)
  p=None
  if count==1:
    if glazer=='NM':
      p=nelder_mead_participant
    elif glazer=='CB':
      p=cobyla_participant
    end=p(participant, start)
    save_fit(end,participant+1,s)
    ret=end


    ret =check(end)
  elif count==2:
    end1=nelder_mead_participant(participant,start)
    end2=cobyla_participant(participant,start)
    dickle={
        "NM": check(end1),
        "CB":(end2)
    }
    save_fit(dickle,participant+1,s)
    ret=dickle
  return ret





In [ ]:
 # works somehow
#dum2=shgo_participant(0) # does not work in time
#dum3=basin_hopping_participant(0) # not converging
#dum2=global_polish(2,'CB')

#b22diff=global_polish(21,"diffev",2,"CB")


In [ ]:
def glass_error(params,participant):
  a=params[0]
  b=params[1]
  l=params[2]
  g=params[3]
  tw=params[4]
  preds=fitPT_TW(participant,a,b,l,g,tw)[0]

  d_plus=0
  d_minus=0
  d0=0
  for day in DAYS:
    vals=info_return(day,participant)

    sold=vals[0]
    pred=preds[day]
    day_err=pred-sold
    if(day_err)>0:
      d_plus+=day_err
    elif(day_err<0):
      d_minus-=day_err

  return d_plus,d_minus


In [ ]:
def stepParam(index, d):
    '''

    model parameters: alpha, beta, lambda, gamma, (tw)

    index: index of the param
    d: boolean, d+ is True, d- is False

    Returns: float for increment value
    '''

    # alpha
    if index == 0:

        print('alpha incremented')
        return -.001 if d else .001
    # beta, lambda, gamma (opposite of alpha)
    elif index == 1 or index == 2 or index==3:
        return stepParam(0, not d)


In [ ]:
def efficient_fit_hardcode_2(participant, guess_,target_loss):
  guess=guess_
  curr_fit = {
    'alpha': guess[0],
    'beta': guess[1],
    'gamma': guess[3],
    'lambda': guess[2],
    'tw': guess[4]
  }


  d_plus, d_minus = glass_error(list(curr_fit.values()), participant)
  f_prime=d_plus+d_minus
  curr_d_plus, curr_d_minus = d_plus, d_minus
  curr_fit['tw'] = 68 # set tw to 68
  done = False
  print(d_plus, d_minus)
  f_d_prime=f_prime

  # TW LOOP
  while curr_fit['tw'] > 1 :
    print(" \n  n\  \n  CURRENT FIT: ", curr_fit.values())
    done=count_errorPT_TW(list(curr_fit.values()), participant)< target_loss
    print("DONE:", done)

    # D+ LOOP

    # lambda loop
    while curr_fit['lambda'] > 1 and curr_fit['lambda'] < 2 and done == False:
      print("D+ done:", done)
      print("CURR_FIT D+", curr_fit.values())

      # gamma loop

      while curr_fit['gamma'] > 0 and curr_fit['gamma'] < 1 and done==False:

        # beta loop
        while curr_fit['beta'] > 0 and curr_fit['beta'] < 1 and done==False:


          # alpha loop
          while curr_fit['alpha'] >= 0 and curr_fit['alpha'] < 1 and done==False:

            curr_fit['alpha'] += stepParam(0,True) #increment alpha
            print("alpha constrained for D+: ", curr_fit['alpha'])

            # check bounds


            # calculate new values for D+ and D-
            curr_d_plus, curr_d_minus = glass_error(list(curr_fit.values()), participant)


            print("curr_d_plus: ", curr_d_plus, "curr d minus: ", curr_d_minus)
            cond=d_plus - curr_d_plus >= 1 and curr_d_minus == d_minus
            print("CONDITION EVER TRUE? : " , cond)


            f_d_prime==curr_d_plus+curr_d_minus # calculate current error
            # CHECK CONDITIONS for improvement
            if d_plus - curr_d_plus >= 1 and curr_d_minus == d_minus:
              # calculate new score

              if f_d_prime < f_prime: # got a better fit, update high score
                f_prime = f_d_prime
                print("new High Score:", f_d_prime)
              elif f_d_prime > f_prime: # starting to get a worse fit
                if(f_d_prime-curr_d_plus >  f_prime): # if this condition is satisfied you are never going to get better fits if you continue moving in this D direction
                  done = True

              d_plus = curr_d_plus

              if d_plus == 0: # improved as much as possible in this direction
                done = True
            # end checking conditions to stop



          # end alpha loop
          curr_fit['beta'] += stepParam(1,True)
          print("beta constrained for D+: ", curr_fit['beta'])


        # end beta loop
        curr_fit['gamma'] += stepParam(3,True)
        print("gamma constrained for D+: ", curr_fit['gamma'])


      # end gamma loop
      curr_fit['lambda'] += stepParam(2,True)
      print("lambda constrained for D+: ", curr_fit['lambda'])


    # end lambda loop


    # D- LOOP (still inside tw loop)



    print("D- Loop starting")
    done=False

    # gamma loop
    while curr_fit['lambda'] > 1 and curr_fit['lambda'] < 2 and done == False:

      print("DONE:::::::::", done)
      print("CURR_FIT D-", curr_fit.values())
      # lambda loop
      while curr_fit['gamma'] > 0 and curr_fit['gamma'] < 1 and done==False:
        # beta loop
        while curr_fit['beta'] > 0 and curr_fit['beta'] < 1 and done==False:
          # alpha loop
          while curr_fit['alpha'] > 0 and curr_fit['alpha'] < 1 and done==False:



            done=count_errorPT_TW(list(curr_fit.values()), participant)< target_loss

            print("alpha constrained for D-: ", curr_fit['alpha'])
            curr_fit['alpha'] += stepParam(0,False)
            print("incremented")


            # check bounds
            if curr_fit['alpha'] >= curr_fit['beta']:
              curr_fit['alpha'] -= stepParam(0,True)


            # calculate new values for D+ and D-
            curr_d_plus, curr_d_minus = glass_error(list(curr_fit.values()), participant)




            print("curr_d_plus: ", curr_d_plus, "curr d minus: ", curr_d_minus)
            cond=d_minus - curr_d_minus >= 1 and curr_d_plus == d_plus
            print("CONDITION EVER TRUE? : " , cond)
            # CHECK CONDITIONS TO STOP if D- has been reduced by 1 (without affecting D-)
            f_d_prime=curr_d_plus+curr_d_minus
            if d_minus - curr_d_minus >= 1 and curr_d_plus == d_plus:
              print("CHECKING CONDITIONS HERE")


              if f_d_prime < f_prime: # got a better fit, update high score
                f_prime = f_d_prime
              elif f_d_prime > f_prime: # starting to get a worse fit
                if(f_d_prime-curr_d_minus >  f_prime):
                  done = True # stop moving in this direction

              d_minus = curr_d_minus

              if d_minus == 0: # improved as much as possible in this direction
                done = True
            # end checking conditions to stop

          # end alpha loop
          curr_fit['beta'] += stepParam(1,False)
          print("beta constrained for D-: ", curr_fit['beta'])


        # end beta loop
        curr_fit['gamma'] += stepParam(3,False)
        print("gamma constrained for D-: ", curr_fit['gamma'])
      # end gamma loop
      curr_fit['lambda'] += stepParam(2,False)
      print("lambda constrained for D-: ", curr_fit['lambda'])

      if curr_fit['alpha'] > curr_fit['beta']:
        curr_fit['alpha'] -= stepParam(0,True)


    # log F' and curr_fit values

    if (d_plus > d_minus):
      break
    else:
      curr_fit['tw'] -= 1
  # end tw loop

  # pick the tw value from the log with the best F' and return it
  pickle.dump(curr_fit, open("/content/drive/MyDrive/Glass Lab Data/PT_TW fits/EfficientFit/participant{part}hardcode".format(part=participant), 'wb'))
  return curr_fit


'''
alpha, beta, gamma, lambda = initial guess
tw = 68
while tw > 1 and D+ < D-: this is tw loop

  D+ LOOP
  while lambda between 1 and 2 AND done is False:
    while gamma between 0 and 1:
      while beta bewteen 0 and 1:
        while alpha between 0 and 1:
          alpha++

          if alpha >= beta:
            set alpha back to previous value and break. incrementing further would go out of bounds

          calculate new values for D+, D-

          CHECK CONDITIONS TO STOP if D has been reduced by 1 (without affecting D-):
            calculate new F" value

            if F" value is better than current high score (aka F'):
              make F" our new high score (set F' = F")
            else if f''- D >f':
              we're starting to get a worse fit, so stop this direction:
              set done to true

            set d_plus to our value for curr_d_plus (calculated before checking conditions)

            if d_plus is now 0, we've improved as much as we can with the current values of beta/gamma/lambda/tw, stop changing alpha
              set done to true
          ~~end checking conditions


        ~~end alpha loop
        beta--

      ~~end beta loop
      gamma--

    ~~end gamma loop
    lambda--

  ~~end lambda loop

  D- LOOP
  set done back to true
  while lambda between 1 and 2 AND done is False:
    while gamma between 0 and 1:
      while beta bewteen 0 and 1:
        while alpha between 0 and 1:
          alpha--

          # don't need to check alpha < beta since alpha decreasing and beta increasing in the D- loop

          calculate new values for D+, D-

          CHECK CONDITIONS TO STOP if D- has been reduced by 1 (without affecting D+):
            calculate new F" value

            if F" value is better than current high score (aka F'):
              make F" our new high score (set F' = F")
            else if f''- D >f':
              we're starting to get a worse fit, so stop this direction:
              set done to true

            set d_plus to our value for curr_d_plus (calculated before checking conditions)

            if d_plus is now 0, we've improved as much as we can with the current values of beta/gamma/lambda/tw, stop changing alpha
              set done to true
          ~~end checking conditions


        ~~end alpha loop
        beta++

      ~~end beta loop
      gamma++

    ~~end gamma loop
    lambda++

  ~~end lambda loop


  log F' and current param values
  tw--

~~end tw loop

pick the tw value from the log with the best F' and return it


F' should now be optimal value
values for alpha, beta, lambda, gamma, tw should now also be optimal
'''

'\nalpha, beta, gamma, lambda = initial guess\ntw = 68\nwhile tw > 1 and D+ < D-: this is tw loop\n\n  D+ LOOP\n  while lambda between 1 and 2 AND done is False:\n    while gamma between 0 and 1:\n      while beta bewteen 0 and 1:\n        while alpha between 0 and 1:\n          alpha++\n\n          if alpha >= beta:\n            set alpha back to previous value and break. incrementing further would go out of bounds\n\n          calculate new values for D+, D-\n\n          CHECK CONDITIONS TO STOP if D has been reduced by 1 (without affecting D-):\n            calculate new F" value\n\n            if F" value is better than current high score (aka F\'):\n              make F" our new high score (set F\' = F")\n            else if f\'\'- D >f\':\n              we\'re starting to get a worse fit, so stop this direction:\n              set done to true\n\n            set d_plus to our value for curr_d_plus (calculated before checking conditions)\n\n            if d_plus is now 0, we\'ve i

In [ ]:


def efficient_fit_hardcode_gpt(participant, guess_, target_loss):
    def update_fit_param(param, increment, is_positive):
        return param + stepParam(increment, is_positive)

    def check_conditions(curr_d_plus, curr_d_minus, d_plus, d_minus, f_prime):
        f_d_prime = curr_d_plus + curr_d_minus
        if curr_d_plus == d_plus and curr_d_minus == d_minus:
            return f_d_prime, f_prime, False

        if d_plus - curr_d_plus >= 1 and curr_d_minus == d_minus:
            if f_d_prime < f_prime:
                return f_d_prime, f_prime, False
            elif f_d_prime > f_prime and (f_d_prime - curr_d_plus > f_prime):
                return f_d_prime, f_prime, True
            return f_d_prime, f_prime, False
        elif d_minus - curr_d_minus >= 1 and curr_d_plus == d_plus:
            if f_d_prime < f_prime:
                return f_d_prime, f_prime, False
            elif f_d_prime > f_prime and (f_d_prime - curr_d_minus > f_prime):
                return f_d_prime, f_prime, True
            return f_d_prime, f_prime, False

        return f_d_prime, f_prime, False

    guess = guess_
    curr_fit = {
        'alpha': guess[0],
        'beta': guess[1],
        'gamma': guess[3],
        'lambda': guess[2],
        'tw': guess[4]
    }

    d_plus, d_minus = glass_error(list(curr_fit.values()), participant)
    f_prime = d_plus + d_minus
    curr_fit['tw'] = 68

    while curr_fit['tw'] > 1:
        done = count_errorPT_TW(list(curr_fit.values()), participant) < target_loss
        if done:
            break

        for param in ['lambda', 'gamma', 'beta', 'alpha']:
            while curr_fit[param] > 0 and curr_fit[param] < 1 and not done:
                curr_fit[param] = update_fit_param(curr_fit[param], {'alpha': 0, 'beta': 1, 'gamma': 3, 'lambda': 2}[param], True)
                curr_d_plus, curr_d_minus = glass_error(list(curr_fit.values()), participant)
                f_prime, _, done = check_conditions(curr_d_plus, curr_d_minus, d_plus, d_minus, f_prime)
                if done:
                    break

        if not done:
            for param in ['lambda', 'gamma', 'beta', 'alpha']:
                while curr_fit[param] > 0 and curr_fit[param] < 1 and not done:
                    curr_fit[param] = update_fit_param(curr_fit[param], {'alpha': 0, 'beta': 1, 'gamma': 3, 'lambda': 2}[param], False)
                    curr_d_plus, curr_d_minus = glass_error(list(curr_fit.values()), participant)
                    f_prime, _, done = check_conditions(curr_d_plus, curr_d_minus, d_plus, d_minus, f_prime)
                    if done:
                        break

        curr_fit['tw'] -= 1

    pickle.dump(curr_fit, open(f"/content/drive/MyDrive/Glass Lab Data/PT_TW fits/EfficientFit/participant{participant}hardcode", 'wb'))
    return curr_fit


In [ ]:
# prompt: run the hardcode efficient fit function

# Assuming necessary data and functions are defined in the preceding code

# Example usage:
result = efficient_fit_hardcode_2(participant=1, guess_=[0.5, 0.5, 1, 0.5, 68], target_loss=4)
result1=efficient_fit_hardcode_gpt(participant=1, guess_=[0.5, 0.5, 1, 0.5, 68], target_loss=4)
print(result,result1)


200 76
 
  n\  
  CURRENT FIT:  dict_values([0.5, 0.5, 0.5, 1, 68])
DONE: False
D- Loop starting
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alpha incremented
alp

In [ ]:
result

{'alpha': 0.5, 'beta': 0.5, 'gamma': 0.5, 'lambda': 1, 'tw': 68}

In [ ]:
x=count_errorPT_TW([.5,.5,1.5,.5,68],1)

KeyboardInterrupt: 

In [ ]:
x

82

In [ ]:
glass_error([.5,.5,1.5,.3,68],3)

In [ ]:
glass_error([.5,.5,1.5,.5,68],3)

In [ ]:
# sdfhudsgyusdfgjcwyjbtkeszdbjhcfwhescdrgfh4wcesjzgfgyug45erfthjgtfjg5e44ysejwfgeuwruqwgydawejerfbcasdhzgahgsfagsdf

In [ ]:
def change(index,d_plus):
  if d_plus:
    if index==0:
      return .001
    else:
      return -.001
  else:
    if (index==0):
      return -.001
    else:
      return .001

In [ ]:
def efficient_fit_ayan_jaiman(participant, target=0, guess=GUESS, maxiter=50000):
  '''
  Finds the best fit using an efficient version of brute force devised by Dr. Glass.

  params:
    participant   :       int             index of the participant
    target        :       int             the optimal count_error value that we want to reach
    guess         :       list<float>     the initial parameter values (usually are the previous participant's best fits)
    maxiter       :       int             maximum number of iterations, to prevent CPU burden


  notes:  what's target for? I thought we improve as much as possible rather than stopping at a set value
          Why do we have a focus_param_index? we are starting with just alpha but we are going to eventually change both alpha and beta at the same time.
          So, best_fit and curr_fit are the *overall* equivalents of f_d_prime and f_prime?
  '''

  # our ultimate best fit starts at the initial param values
  ultimate_best_fit = guess

  # find the count_error we are starting with (this is our f_prime)
  f_prime = count_errorPT_TW(ultimate_best_fit,participant)

  # to track iterations
  i = 0

  # for fitting each parameter sequentially
  curr_fit = [guess].pop()

  # to log our parameter values
  visited = []


  # MAIN OPTIMIZATION LOOP
  while(f_prime>target or i<maxiter ):

    # set the focus parameter (starting with alpha) (see notes)
    focus_param_index = 0

    # record our baseline value for the parameter
    og_baseline_param = ultimate_best_fit[focus_param_index]

    # update f_prime
    f_prime = count_errorPT_TW(ultimate_best_fit,participant)

    # ??
    best_fit=ultimate_best_fit


    # QUESTION is this to skip over parameter values we have seen before? can we discuss this
    if curr_fit in visited:
      continue

    # calculate our D+ and D- values (what must be reduced)
    d_plus,d_minus = glass_error(curr_fit, participant)
    curr_d_plus = d_plus
    curr_d_minus = d_minus

    # ??
    baseline_param = og_baseline_param

    # DECLARE OUR F"
    f_d_prime = f_prime

    # OPTIMIZING D+
    while (f_d_prime - d_plus < f_prime and d_plus > 0): # our D+ condition

      # THIS IS WHILE LOOP #2
      while (d_plus - curr_d_plus < 1): # First, change alpha to reduce D+ by 1

        # increment our baseline alpha value
        baseline_param -= .001  # decrease alpha -> smaller predicted values -> decreased D+

        # update curr_fit with our new alpha value
        curr_fit = np.array( curr_fit[:focus_param_index] + [baseline_param] + curr_fit[focus_param_index:] ) # [baseline_param]+list(curr_fit)[~focus_param_index])

        # calculate our new F", D+, D-
        f_d_prime = count_errorPT_TW(curr_fit,participant)
        curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

        # log our new parameter values
        visited.append(curr_fit)

      # END WHILE LOOP #2

      # Now that D+ has been reduced, D- may or may not have been increased (since predicted values decreased)
      # So if D- hasn't increased, we have a better fit
      if (curr_d_minus <= d_minus):
        best_fit = curr_fit
        if (f_d_prime < f_prime):
          f_prime = f_d_prime # best error after step 1 is the error reached if d_minus doesn't change

      else:
        # if D- has been affected, do NOT update f_prime
        continue


      d_plus=curr_d_plus
      d_minus=curr_d_minus

  # END D+ OPTIMIZATION


  # OPTIMIZING D-
  while (f_d_prime - d_minus < f_prime and d_minus > 0): # our D- condition


    # THIS IS WHILE LOOP #2
    while (d_minus - curr_d_minus < 1): # First, change alpha to reduce D- by 1

      # increment our baseline alpha value
      baseline_param += .001  # increase alpha -> greater predicted values -> decreased D-

      # update curr_fit with our new alpha value
      curr_fit=np.array( curr_fit[:focus_param_index] + [baseline_param] + curr_fit[focus_param_index:] ) # [baseline_param]+list(curr_fit)[~focus_param_index])

      # calculate our new F", D+, D-
      f_d_prime = count_errorPT_TW(curr_fit,participant)
      curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

      # log our new parameter values
      visited.append(curr_fit)

    # END WHILE LOOP #2

    # Now that D- has been reduced, D+ may or may not have been increased (since predicted values decreased)
    # So if D+ hasn't increased, we have a better fit
    if (curr_d_plus <= d_plus):
      best_fit = curr_fit
      # sanity check
      if (f_d_prime < f_prime):
        f_prime = f_d_prime # best error after step 1 is the error reached if D+ doesn't change
    else:
      # if D- has been affected, do NOT update f_prime
      continue


    d_plus = curr_d_plus
    d_minus = curr_d_minus

  # END D- OPTIMIZATION


  # NOW WE ARE LOOKING AT BETA AND ALPHA TOGETHER
  focus_param_index = 1
  baseline_param = curr_fit[focus_param_index]


  # STEP 2 STEP 2 STEP 2 STEP 2
  # idea is to start from the middle of the alpha search space

  while (curr_fit[1] < 1): # incrementing beta

    while (curr_fit[0] > 0): # decrementing alpha
      curr_fit[0] -= .001

      # continue if alpha < beta
      if (curr_fit[0] < curr_fit[1]):
        continue

      # calculate our new F", D+, D-
      curr_d_plus,curr_d_minus = glass_error(curr_fit,participant)

      if (d_plus - curr_d_plus > 1):
        f_d_prime = count_errorPT_TW(curr_fit,participant)
        if (f_d_prime < p_prime):
          f_prime = f_d_prime
        d_plus = curr_d_plus

    curr_fit[1] += .001


  # STEP 3 STEP 3 STEP 3






In [2]:
def efficientFit_sidney(parameterNum, participant, guess):
    
    #Finds the best fit using an efficient version of brute force devised by Dr. Glass.

    #params:
        #parameterNum  :       int             index of parameter being fitted [alpha, beta, lambda, gamma]
        #participant   :       int             index of the participant
        #guess         :       list<float>     the initial parameter values (usually are the previous participant's best fits)


    #dependant variables:
        #bestFit : referes to the parameters of the fit that produces minDev, f'
        #currFit : referes to the parameters of the current fit being tested, f''
        #(bestDevPlus, bestDevMinus) & (currDevPlus, currDevMinus) : refers to D+ and D- produced by the respective parameters of the fits

    #notes:
        #Complete for a single participant as a recursive function
        #increasing alpha increases D-, decreasing alpha increases D+
        #increasing beta, lambda, gamma increases D+, decreasing increases D-
        #in order to decrease D+ or D-, we move in the direction that we move in the opposite direction which we know increases it i.e. to decrease D-, we decrease alpha as we we know doing the opposite will only increase D-

    if parameterNum == -1: return guess #base case

    bestFit = guess.copy()
    currFit = bestFit.copy()

    changeMultiplier = 1

    if parameterNum == 0: changeMultiplier = -1 

    bestDPlus, bestDMinus = glass_error(bestFit, participant)
    currDPlus, currDMinus = bestDPlus, bestDMinus

    stepSize = 0

    while (sum(glass_error(bestFit, participant)) >= currDMinus): #sum(glass_error(bestFit, participant)) is (D+)+(D-)=f', currDminus is f''-D+
        while (currDPlus > (bestDPlus - 1)):
            currFit[parameterNum] -= stepSize*changeMultiplier*.001
            currFit = efficientFit_sidney(parameterNum-1, participant, currFit)
            currDPlus, currDMinus = glass_error(currFit, participant)
            stepSize += 1
        
        if ((currDPlus+currDMinus) < (bestDPlus+bestDMinus)):
            bestFit = currFit.copy()
            bestDPlus, bestDMinus = currDPlus, currDMinus

    stepSize = 0
    currFit = bestFit.copy()
    currDPlus, currDMinus = bestDPlus, bestDMinus
    

    while (sum(glass_error(bestFit, participant)) >= currDPlus): #sum(glass_error(bestFit, participant)) is (D+)+(D-)=f', currDplus is f''-D-
        while (currDMinus > (bestDMinus - 1)):
            currFit[parameterNum] += stepSize*changeMultiplier*.001
            currFit = efficientFit_sidney(parameterNum-1, participant, currFit)
            currDPlus, currDMinus = glass_error(currFit, participant)
            stepSize += 1
        
        if ((currDPlus+currDMinus) < (bestDPlus+bestDMinus)):
            bestFit = currFit.copy()
            bestDPlus, bestDMinus = currDPlus, currDMinus

    
    return bestFit

Glass questions


In [ ]:
def exhaustive_fit_with_guess_one_subject(participant,precision,guess,target):
                                #subject:        int,

        """Performs the exhaustive fit algorithm for one subject and saves the best fit.
        The exhaustive fit algorithm consists of iterating through all possible parameter values (with a given level of precision) and accepting the best.
        Inputs:
            subject: the participant's number in the dataframe.
            precision: the amount to increment each value when iterating through all possible values.
            verbose: set to True to get progress bars for the fitting.
            error_type: should the error be calculated as the absolute difference
                        between the prediction and the amount, or as
                        the difference in proportion of goods sold. report.docx
                        seems to use proportional.
            tw: if provided, tw will stay at this constant level."""

        #prev_best_fit: Parameters = self.best_fits[subject]
        #initial_pred = self.predict_one_subject(subject=subject, fit=prev_best_fit)
        #lowest_error = self.mean_error_one_subject_proportion(subject, initial_pred)
        prev_best_fit=guess #array/list
        best_fit = prev_best_fit #array/ list
        initial_pred=fitPT_TW(participant,guess[0],guess[1],guess[2],guess[3],guess[4])[0] #a,b,l,g,tw

        # Set lowest error to initial error

       #lowest_error=fit_errorPT_TW(prev_best_fit,participant)  # d2a error
        lowest_error=count_errorPT_TW(prev_best_fit,participant)
        # Get lists of all possible values for all free params
        low  = lambda fit, old_precision, floor: max(floor, fit - old_precision/2) if fit else 0  #basically setting the floor which is 0.01,
        high = lambda fit, old_precision, ceil:  min(ceil,  fit + old_precision/2) if fit else 0  # setting the ceil which is 1 for except lam which it is 2
        valid_parameter_ranges= {                        # Dict[str, List[float]] = {

            "a": list(np.arange(
                            .01,#low(prev_best_fit[0],  prev_precision, precision), # here he is setting the minimum value to the value of the guess or the floor value which
                             1, #high(prev_best_fit[0], prev_precision, 1),# + EPS,
                            precision
                )),
            "b": list(np.arange(
                            .01,#low(prev_best_fit[1],  prev_precision, precision),
                            1,  #high(prev_best_fit[1], prev_precision, 1),# + EPS,
                            precision
                )),
            "l": list(np.arange(
                            1,#low(prev_best_fit[2],  prev_precision, 1),
                            2,#high(prev_best_fit[2], prev_precision, 2),# + EPS,
                            precision
                )),

            "g": list(np.arange(
                            .01,#low(prev_best_fit[3],  prev_precision, precision),
                             1,#high(prev_best_fit[3], prev_precision, 1),# + EPS,
                            precision
                )),

            "tw": list(np.arange(2, 68, 1))
        }
        #if tw is not None:
         # valid_parameter_ranges["tw"] = [tw]

        # Remove data on non-free params:
      #  ranges: List[List[Any]] = [valid_parameter_ranges[param] if param in self.free_params
         #                                                        else [None]
         #                                                        for param in PARAM_LIST ]
        ranges=[valid_parameter_ranges[param] for param in PARAM_LIST]
        # Get all possible values via cartesian product
        all_possible_fits = list(product(*ranges)) # takes every possible range

        # Iterate through every possible value
        iterations = 1
        for range in ranges:
            iterations *= len(range)
       # for fit in tqdm(all_possible_fits,

                      #  leave=False,
                       # total=iterations,
                       # desc="Attempting all fits..."):
        count=0
        for fit in all_possible_fits:
            count+=1
            print("iter:",count)
            if (lowest_error<=target) or (count>MAX_ITER):
              break
            # Skip fits where a > b
            if fit[0] > fit[1]:
                continue
            # Keep fits where xl-.1 <= xg <= xl
         #   if (fit[2] is not None and fit[3] is not None) and (fit[2] > fit[3] or fit[2] < fit[3]-0.1):
               # continue

            # Predict sale amounts based on fit

           # predictions: List[int] = self.predict_one_subject(subject, fit_params)
            fit_params=fit

            preds=fitPT_TW(participant,fit_params[0],fit_params[1],fit_params[2],fit_params[3],fit_params[4])
            # Get error for the given fit
            #error: float = error_fn(subject, predictions)
            #error=fit_errorPT_TW(np.array(fit_params),participant)
            error=count_errorPT_TW(np.array(fit_params),participant)
            # Check if it's the best so far:
            if error < lowest_error:
                lowest_error = error
                best_fit  = fit_params

            elif error == lowest_error:
                if fit_params[0] is not None and fit_params[0] >= best_fit[0]:
                    best_fit = fit_params
                elif not fit_params[0]:
                    best_fit = fit_params
                #self.all_best_fits[subject].append(fit_params)

        save_fit(best_fit,participant+1,"brute")
        return best_fit



In [ ]:
ranges=[(0,1), (0,1), (1,2), (0,1), (2,68)]

In [ ]:

def get_neighbors(params,precision):
  params=list(params)
  neighbuurs=[]


  for index,p in enumerate(params):
    add=precision
    if index==4:
      add=1
    bound_up=lambda p,i: p if p+add>=ranges[i][1] else p+add
    bound_down=lambda p,i: p if p-add<=ranges[i][0] else p-add


    poop=params.copy()
    up_n=poop.copy()
    up_n[index]=bound_up(p,index)
    down_n=poop.copy()
    down_n[index]=bound_down(p,index)
    neighbuurs.append(up_n)
    neighbuurs.append(down_n)
# ret=[np.array(l) for l in neighbuurs]

  return ret




In [ ]:
def bfs_fit_one_subject(participant, guess, target, precision



):


        # Ensure greedy fit has a starting point

        # Set correct error function
        #error_fn = self.get_error_fn(error_type)
        error_fn=count_errorPT_TW

                # Get initial error

        start_error = error_fn(guess,participant)
        best_fit=guess
        best_error=start_error

        # Keep track of every visited node, along with its error
        #visited: Dict[Parameters, bool] = {}
        visited={} #(np.array,error) : bool

        # Keep track of the current best candidate
        lowest_error = start_error

        # BFS keeps track of a queue of nodes to be visited, and checks the oldest first
        #bfs_queue: List[Tuple[Parameters, float]] = [(start_fit, float('inf'))]
        bfs_queue=[(guess,float('inf'))]
        iter=0

        #while len(bfs_queue) != 0:
        while iter<1000 and lowest_error>target:
            # Get current node
            #print(f'{len(bfs_queue)} nodes remaining / {len(visited)} visited', end="\r")
            current_fit, parent_error = bfs_queue.pop(0)

            if visited.get((tuple(current_fit),parent_error)): #If you've already been here...
                continue

              # marking your spot

            #predictions = self.predict_one_subject(subject, current)
            error = error_fn(current_fit, participant)
            visited[(tuple(current_fit),error)] = True

            if error > parent_error:
                continue

            if error < lowest_error:
                lowest_error = error
                best_fit = current_fit
               # self.all_best_fits[subject] = [current]
            elif error == lowest_error:
                best_fit=current_fit


            all_neighbors = get_neighbors(current_fit, precision)

            # Iterate through each neighbor
            best_n=100
            for neighbor in all_neighbors:

                # Get fit as parameter
                #neighbor_fit = Parameters(*neighbor)
                # If fit uses a and b, a must be less than b
                if neighbor[0] is not None and neighbor[1] is not None and neighbor[0] > neighbor[1]:
                    continue
                # Skip visited nodes
                if visited.get(tuple(neighbor)):
                    continue
                n_error=error_fn(neighbor, participant)
                visited[(tuple(neighbor),n_error)]=True
                if n_error<best_n:
                  bfs_queue.append((neighbor, error))
                # Get predictions and errors for each neighbor
        save_fit(best_fit, participant, "neighbors")
        return best_fit



In [ ]:
#bf23=exhaustive_fit_with_guess_one_subject(22,.1,np.array([.5,.5,1.5,.5,50]),28)
#print("official fit:     "+bf23)


In [ ]:
#bf23

In [ ]:
#bf23=direct_participant(22)

In [ ]:
#bf23=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Exhuastive fits/participant 23 December 10, 2023.pickle')

In [ ]:
def takeSecond(elem):
  return elem[1]

In [ ]:
def sorted_ranking(lst):
  entire_tuple_list = []
  for i in range (len(lst)):
    tempTup = (i,count_error(lst[i],i))
    entire_tuple_list.append(tempTup)
  entire_tuple_list.sort(key=takeSecond)
  return entire_tuple_list

In [ ]:
import os

In [ ]:
fit_data=os.listdir('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Diffy evolution fits')

In [ ]:
fit_data[1][12:14]

In [ ]:
fit_data

In [ ]:
diffev_fits=[(int(path[12:14]),load_fit(DIFEV_SAVE_PATH+path)) for path in fit_data]

In [ ]:
df_days

In [ ]:
diffev_fits

In [ ]:
def sort_tuples(lst):
    return sorted(lst, key=lambda x: x[0])


In [ ]:
pt_tw_errors=[(i-1,count_errorPT_TW(fit,i-1)) for (i,fit) in diffev_fits]

In [ ]:
pt_tw_errors

In [ ]:
sorted_errors=sort_tuples(pt_tw_errors)

In [ ]:
len(sorted_errors
    )

In [ ]:
sorted_errors[33]

In [ ]:
sorted_errors

In [ ]:
sorted_errors.remove(sorted_errors[24])
sorted_errors.remove(sorted_errors[33])
sorted_errors.remove(sorted_errors[6])


In [ ]:
our_errors=pd.read_csv('/content/drive/MyDrive/Glass Lab Data/our_errors - fixed our_errors.csv')

In [ ]:
our_errors

In [ ]:
our_errors=our_errors.drop(["Unnamed: 0"], axis=1)

In [ ]:
our_errors

In [ ]:
z=zip(list(our_errors.index),list(our_errors['our_error'].values))

In [ ]:
PT_errs=list(z)

In [ ]:
PT_errs

In [ ]:
sorted_errors

In [ ]:
PT_better=[]
PT_TW_better=[]
equals=[]
for i in range(57):
  Pt_tw=sorted_errors[i]
  Pt=PT_errs[i]
  if Pt_tw[1]<Pt[1]:
    PT_TW_better.append(i)
  elif Pt_tw[1]==Pt[1]:
    equals.append(i)
  else:
    PT_better.append(i)


In [ ]:
PT_better

In [ ]:
bf22=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/participant 22 December 27, 2023.pickle')

In [ ]:
count_errorPT_TW(bf22['CB'],21)

ALL OF THE ABOVE MUST BE CHANGED

improved pt_tw

participant 2 (i=1) fit=
array([ 0.40763567, 0.43357354, 1.03015975, 0.96945102, 14.00001488])
count error=1


In [ ]:
loaded_bf23_dif= load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Diffy/participant 23 January 02, 2024.pickle')
loaded_bf23_dir=load_fit('/content/drive/MyDrive/Glass Lab Data/PT_TW fits/Polished_Direct/participant 23 January 02, 2024.pickle')


In [ ]:
loaded_bf23_dif

In [ ]:
count_errorPT_TW(loaded_bf23_dir['NM'],22)

In [ ]:
loaded_bf23_dir

In [ ]:
lowest_error=100
best_tw=0
for tw in range(2,68):
  err=count_errorPT_TW([1.0 , 1, 1,.355,tw],21)
  if err<lowest_error:
    best_tw=tw
    lowest_error=err


In [ ]:
lowest_error

In [ ]:
best_tw

In [ ]:
count_errorPT_TW([1,1,1,.35,61],21)

In [ ]:
PT_TW_better

In [ ]:
equals